In [1]:
from sklearn import datasets, svm, metrics
from sklearn.utils import resample
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from src import confidence_planner

digits = datasets.load_digits()

# Flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

n_iter = 100
accuracies = []

# Run model training and testing n_iter times
for i in range(n_iter):
    # Bootstrap indices
    indices = resample(range(n_samples), n_samples=n_samples)

    X_train = []
    y_train = []
    X_test = []
    y_test = []

    # Append images and labels with bootstrapped indices to lists for training
    for idx in indices:
        X_train.append(data[idx])
        y_train.append(digits.target[idx])

    # Append images and labels that are not in a training subset to lists for testing
    for i in range(n_samples):
        if i not in indices:
            X_test.append(data[i])
            y_test.append(digits.target[i])

    # Learn the digits on the train subset
    clf.fit(X_train, y_train)

    # Predict the value of the digit on the test subset
    predicted = clf.predict(X_test)

    # Get the model's accuracy in the form of percentages
    accuracy_percents = 100*metrics.accuracy_score(y_test, predicted)

    # Append accuracy to the list
    accuracies.append(accuracy_percents)

# Print confidence interval around the obtained accuracy with 85% confidence
print(confidence_planner.percentile_BM(accuracies, 0.85))

Confidence intervals around given accuracy for your confidence and 90%, 95%, 98%, 99%.
([98.34797263022624, 99.39236881005174], [98.29609577202015, 99.40227035983271], [98.16460611658026, 99.53831135578031], [98.07967819276, 99.54766938686795], [98.07544740498534, 99.55549408208026])
